In [23]:
!pip install gradio tensorflow pandas_datareader requests

In [27]:
# ── Install dependencies ───────────────────────────────────────────────────────
!pip install --quiet gradio tensorflow pandas requests matplotlib

# ── Imports ────────────────────────────────────────────────────────────────────
import requests, zipfile, io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import Input
import gradio as gr

# ── 1. Download & load World Bank unemployment data ────────────────────────────
url = 'http://api.worldbank.org/v2/en/indicator/SL.UEM.TOTL.ZS?downloadformat=csv'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
fname = [n for n in z.namelist() if n.endswith('.csv') and 'Metadata' not in n][0]
df = pd.read_csv(z.open(fname), skiprows=4)

# ── 2. Clean & reshape data ────────────────────────────────────────────────────
year_cols = [col for col in df.columns if str(col).isdigit()]
df_long = df.melt(
    id_vars=['Country Name','Country Code'],
    value_vars=year_cols,
    var_name='Year',
    value_name='UnemploymentRate'
).dropna(subset=['UnemploymentRate'])
df_long['Year'] = df_long['Year'].astype(int)

# ── 3. Compute global average unemployment rate per year ───────────────────────
global_ts = df_long.groupby('Year')['UnemploymentRate'].mean().reset_index()

# ── 4. Prepare data for LSTM model ─────────────────────────────────────────────
window_size = 5
data = global_ts['UnemploymentRate'].values.reshape(-1,1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

X, y = [], []
for i in range(len(data_scaled) - window_size):
    X.append(data_scaled[i:i+window_size, 0])
    y.append(data_scaled[i+window_size, 0])
X, y = np.array(X), np.array(y)
X = X.reshape(X.shape[0], X.shape[1], 1)

# ── 5. Build and train LSTM model ──────────────────────────────────────────────
model = Sequential([
    Input(shape=(window_size, 1)),
    LSTM(32),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, batch_size=4, verbose=0)

# ── 6. Helper Functions ────────────────────────────────────────────────────────
def predict_future(year):
    last_known = global_ts['Year'].max()
    vals = list(global_ts['UnemploymentRate'].values)
    scaled = list(data_scaled.flatten())
    preds = []
    years = list(global_ts['Year'])

    for y in range(last_known+1, year+1):
        window = np.array(scaled[-window_size:]).reshape(1, window_size, 1)
        pred = model.predict(window, verbose=0)[0,0]
        scaled.append(pred)
        val = scaler.inverse_transform([[pred]])[0,0]
        vals.append(val)
        preds.append(val)
        years.append(y)

    return years, vals

def top_increases(y1, y2, top_n=10):
    df1 = df_long[df_long['Year']==y1].set_index('Country Name')['UnemploymentRate']
    df2 = df_long[df_long['Year']==y2].set_index('Country Name')['UnemploymentRate']
    common = df1.index.intersection(df2.index)
    delta = (df2[common] - df1[common]).sort_values(ascending=False)
    result = pd.DataFrame({
        'Country': delta.index,
        f'Rate in {y1}': df1[common].loc[delta.index].values,
        f'Rate in {y2}': df2[common].loc[delta.index].values,
        'Increase': delta.values
    })
    return result.head(top_n)

# ── 7. Gradio Interface Function ───────────────────────────────────────────────
def gradio_fn(year1, year2):
    if year1 >= year2:
        return "⚠️ Year 2 must be greater than Year 1.", pd.DataFrame(), None, None

    # Global prediction
    years, vals = predict_future(year2)
    pred1 = round(vals[years.index(year1)], 2)
    pred2 = round(vals[years.index(year2)], 2)

    # Plot global trend
    fig1, ax1 = plt.subplots()
    ax1.plot(years, vals, marker='o')
    ax1.set_title("📈 Global Unemployment Rate Trend")
    ax1.set_xlabel("Year")
    ax1.set_ylabel("Unemployment Rate (%)")
    ax1.grid(True)

    # Bar chart for top countries
    inc_df = top_increases(year1, year2)
    fig2, ax2 = plt.subplots()
    ax2.barh(inc_df['Country'], inc_df['Increase'], color='salmon')
    ax2.set_title("🌍 Top 10 Country Increases")
    ax2.set_xlabel("Increase in Unemployment Rate (%)")
    ax2.invert_yaxis()

    # Result text
    text = (
        f"📊 Predicted Global Unemployment:\n"
        f"• {year1}: {pred1:.2f}%\n"
        f"• {year2}: {pred2:.2f}%\n\n"
        f"🔝 Top 10 Countries with Highest Increase:"
    )
    return text, inc_df, fig1, fig2

# ── 8. Build Interface ─────────────────────────────────────────────────────────
year_min = df_long['Year'].min()
year_max = df_long['Year'].max()

inputs = [
    gr.Slider(minimum=year_min, maximum=year_max - 1, step=1, label="Year 1"),
    gr.Slider(minimum=year_min + 1, maximum=year_max + 10, step=1, label="Year 2")
]

outputs = [
    gr.Textbox(label="Summary"),
    gr.Dataframe(label="Top Country Increases"),
    gr.Plot(label="Global Trend Graph"),
    gr.Plot(label="Country Bar Chart")
]

# ── 9. Launch Interface ────────────────────────────────────────────────────────
gr.Interface(
    fn=gradio_fn,
    inputs=inputs,
    outputs=outputs,
    title="🌐 Global Unemployment Rate Predictor",
    description="Select two years to view predicted global unemployment trends and countries with the highest rate increase.",
    theme="default"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82cd747b98cf641807.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
